In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini text - Single-turn

## Set configuration

### Package Install
* [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)

In [2]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Authentication to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.
* Refer to the [authentication ways](https://cloud.google.com/docs/authentication?hl=ko) in GCP

In [3]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define constants

In [16]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash-002" # @param {type:"string"}

### Import libraries

In [17]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Tool,
    Part,
    ChatSession
)

from vertexai.preview.generative_models import grounding

### Initalize Vertex AI

In [18]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=LOCATION)

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME)

### Helper function

In [19]:
def generate(query:str,
             stream:bool) -> GenerationResponse:

    """
    Generate a response from the model.

    Args:
      query : query string to generate response
      stream : whether to stream the response

    Returns:
      GenerationResponse: The generated response.

    """

    # Request body
    #   https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#request
    #   https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = GenerationConfig(
        temperature=0.9,
        top_p=1.0,
        top_k=32,
        candidate_count=1,
        max_output_tokens=8192,
    )

    contents = [query]
    # Google Search grounding.
    tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

    # Counts tokens
    print(f"Token Count : {model.count_tokens(contents)}")

    # Configure satey setting
    # : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    }

    responses = model.generate_content(
        contents,
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=stream,
        tools=[tool]
    )

    return responses


### Execution

In [20]:
from time import perf_counter

t1_start = perf_counter()

prompt = """
        당신은 사람들이 AI 개념을 이해하도록 돕는 생성 AI 전문가입니다.
        아래 내용에 대해서 상세하게 알려주세요.

        1. 최신 생성형 AI 트랜드에 대해서 설명해주세요.
        2. 올해 12월에 생성형 AI 주요 뉴스(날짜 포함)에 대해서 10가지 항목으로 요약해주세요.
        3. 생성형 AI의 2025년도 중요한 주제는 어떤것인가요 ?
"""

stream = False

responses = generate(prompt, stream)

if stream:
  for response in responses:
    print(response.text, end="")
else:
  # Print the model response
  print(f'\n Usage metadata:\n{responses.to_dict().get("usage_metadata")}')
  print(f"\n Finish reason:\n{responses.candidates[0].finish_reason}")
  print(f"\n Safety settings:\n{responses.candidates[0].safety_ratings}")

  display(Markdown(responses.text))

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")


Token Count : total_tokens: 126
total_billable_characters: 140


 Usage metadata:
{'prompt_token_count': 126, 'candidates_token_count': 1873, 'total_token_count': 1999}

 Finish reason:
1

 Safety settings:
[category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
probability_score: 0.00460956944
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0321006887
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
probability_score: 0.0136368303
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0362200849
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
probability_score: 0.00912565179
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0460338742
, category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
probability_score: 0.0140636312
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0356785394
]


## 생성형 AI에 대한 상세 설명

### 1. 최신 생성형 AI 트렌드

최근 생성형 AI는 폭발적인 성장세를 보이며 여러 분야에서 활용되고 있습니다.  초기의 과장된 기대와 달리, 현재는 AI의 한계와 비용에 대한 현실적인 인식이 자리 잡았습니다.  기업들은 AI를 단순한 직원 대체 수단이 아닌, 기존 인력을 보완하고 생산성을 높이는 도구로 인식하고 있습니다.  

주요 트렌드는 다음과 같습니다.

* **AI와 인간의 협력:**  완전 자동화보다는, 인간의 감독과 개입이 필수적인 AI와의 협업 모델이 주목받고 있습니다. AI는 데이터 처리와 인사이트 제공에 집중하고, 중요한 의사결정은 인간이 담당하는 방식입니다.
* **신뢰 및 규정 준수:** AI가 생성하는 정보의 정확성과 신뢰성에 대한 우려가 커지면서, 신뢰할 수 있는 AI 플랫폼과 규정 준수 방안에 대한 관심이 증가하고 있습니다.  가짜 뉴스나 조작된 콘텐츠 생성 방지에 대한 기술적, 제도적 노력이 필요합니다.
* **다양한 모달 지원:** 텍스트뿐만 아니라 이미지, 음성, 영상 등 다양한 형태의 데이터를 처리하고 생성하는 다중 모달 AI 모델이 발전하고 있습니다. 이는 AI의 활용 범위를 넓히고, 더욱 풍부하고 현실적인 경험을 제공하는 데 기여합니다.
* **개인 맞춤형 경험:** 사용자의 선호도와 필요에 따라 맞춤형 콘텐츠와 서비스를 제공하는 AI 시스템이 발전하고 있습니다.  이는 사용자 만족도를 높이고, AI 기술의 효용성을 증대시키는 데 중요한 요소입니다.
* **지속 가능성:** AI 시스템의 에너지 소비량과 환경 영향에 대한 우려가 커지면서, 환경 친화적인 AI 개발과 운영에 대한 관심이 높아지고 있습니다. 에너지 효율적인 AI 모델 개발과 지속 가능한 데이터 센터 운영이 중요해지고 있습니다.
* **AI 에이전트:** AI가 단순히 명령을 수행하는 도구를 넘어, 사용자의 목표를 이해하고 자율적으로 작업을 수행하는 에이전트로 발전하고 있습니다.  이러한 에이전트는 사용자의 생산성을 향상시키고, 더욱 효율적인 업무 환경을 구축하는 데 기여할 것으로 예상됩니다.
* **AI 리터러시의 중요성:** AI를 효과적으로 활용하고 비판적으로 평가하는 능력이 중요해지고 있습니다.  AI 리터러시 교육과 인증 시스템이 확대될 것으로 예상됩니다.


### 2. 2024년 12월 생성형 AI 주요 뉴스 (10가지)

2024년 12월 생성형 AI 관련 주요 뉴스는 다음과 같습니다.  (날짜는 소스에 명시된 날짜를 기준으로 하며, 실제 뉴스 발표일과 다를 수 있습니다.)

1. (2024-12-25) 로봇 산업과 생성형 AI의 융합을 통한 퀀텀 점프 가능성 제시.
2. (2024-12-24) 생성형 AI의 영향으로 미디어 및 디자인 분야의 채용 공고 감소.
3. (2024-12-24) 구글의 Gemini 2.0 발표와 2025년 AI 시장 주도권 경쟁 전망.
4. (2024-12-23) 삼성SDS 인사이트리포트: 2025년 생성형 AI 트렌드 전망 발표.  AI는 직원 보완, 비용 절감, 혁신 등에 기여할 것으로 예상.
5. (2024-12-23) 팀뷰어의 2025년 AI 전망 발표: 완전 자동화보다는 인간과 AI의 협력에 초점.
6. (2024-12-22)  AI의 에너지 소비 및 환경적 영향에 대한 논의 증가.  AI의 지속 가능성이 중요한 이슈로 부상.
7. (2024-12-16) 오픈AI, 챗GPT 검색 기능을 모든 사용자에게 무료로 개방. 구글 검색 시장에 대한 도전장.
8. (2024-12-12) 생성형 AI를 활용한 사이버 보안 기술의 발전과 사이버 공격 위협 증가.
9. (2024-12-10) 국세청, AI 기반 사이버 보안관제 시스템 구축. AI 해킹 공격 방어.
10. (2024-12-09) 마이크로소프트, 2025년 AI 트렌드 전망 발표.  전 세계 조직의 AI 도입률 증가.


### 3. 생성형 AI의 2025년 중요 주제

2025년 생성형 AI의 중요 주제는 다음과 같이 예상됩니다.

* **AI 에이전트의 발전:**  더욱 자율적이고 복잡한 작업을 수행하는 AI 에이전트의 등장과 상용화.  다양한 AI 시스템의 상호 작용을 최적화하는 '슈퍼 에이전트'의 출현 가능성.
* **신뢰성과 책임성:**  AI가 생성하는 정보의 정확성, 투명성, 공정성에 대한 심도 있는 논의와 규제 마련.  AI의 윤리적 사용과 책임 있는 개발에 대한 관심 증가.
* **지속 가능성:**  AI 시스템의 에너지 소비 감소와 환경 영향 최소화를 위한 기술적, 정책적 노력.  탄소 중립적인 AI 개발과 운영에 대한 관심 고조.
* **인간-중심 AI:** AI가 인간의 능력을 보완하고 삶의 질을 향상시키는 데 기여하도록 하는 인간 중심적인 AI 설계 및 개발. 사용자 친화적인 인터페이스와 편리한 사용성 확보.
* **AI 리터러시 교육:** AI 기술을 이해하고 효과적으로 활용하는 능력을 배양하기 위한 교육 및 훈련 프로그램 확대. AI 활용 능력을 향상시키는 교육과정 개발.
* **산업별 AI 활용:**  각 산업 분야에 특화된 AI 솔루션 개발 및 적용.  제조, 의료, 금융 등 다양한 분야에서의 AI 활용 사례 증가와 그에 따른 경쟁력 강화.
* **AI와 사이버 보안:** AI 기반 사이버 보안 기술 발전과 AI를 악용한 사이버 공격에 대한 대응 방안 마련. AI를 활용한 공격 및 방어 기술의 발전과 그에 대한 대비책 마련.
* **AI 경제학:** AI 기술 발전에 따른 경제적, 사회적 영향 분석과 대응 전략 수립. 일자리 변화, 경제 성장, 사회적 불평등 등 AI가 미치는 영향에 대한 심도있는 연구 및 전망.


이러한 주제들은 상호 연관되어 있으며, 2025년 생성형 AI의 발전 방향과 사회적 영향에 중요한 역할을 할 것으로 예상됩니다.  이는 오늘(2024년 12월 25일) 기준으로 예상되는 내용이며, 앞으로의 기술 발전 추이에 따라 변동될 수 있습니다.


Time : 14.172153955000113 seconds


